In [23]:
import requests #輸入HTTP請求模組：讀取網站檔案
import random #輸入隨機模組
import csv #輸入CSV模組

#----------------------------下方開始為主要程式碼之前置定義函式--------------------------------------------

class FbLottery(object): #物件導向，將Fblottery置放於object類別之下
    def __init__(self, page_url, token): #進行類別初始化
        self.graph_api_url = 'https://graph.facebook.com/v12.0' #GraphAPI位置
        self.access_token = token #權杖位置
        self.page_url = page_url #粉絲專頁網址
        self.page_id = self.get_page_id() #粉絲專頁專屬id
        self.retrieve_limit = 80 #留言抽出之粉絲數量

    def get_page_id(self): #定義get_page_id，並以self為參數
        url = self.graph_api_url + '/{0}/?access_token={1}'.format(
            self.page_url, self.access_token)
        data = requests.get(url).json()
        return data['id'] #取得GraphAPI中'粉絲專頁id'資料

    def get_page_posts(self): #定義get_page_posts，並以self為參數
        url = self.graph_api_url + '/{0}/posts?access_token={1}'.format(
            self.page_id, self.access_token)
        data = requests.get(url).json()
        return data['data'] #取得GraphAPI中'data'資料

    def get_lottery_post(self, title): #定義get_lottery_posts，並以self為參數,title參數為文章關鍵字
        all_posts = self.get_page_posts() #all_posts=取得全部貼文
        for post in all_posts: #for迴圈，尋找目標貼文
            if title in post['message']:  #尋找GraphAPI中'message'資料中含有之關鍵字貼文
                return post #取得之後，回傳該篇貼文至post當中

    def get_post_comments(self, post_id): #定義get_post_comments，並以self為參數,post_id參數為目標貼文id
        url = self.graph_api_url + '/{0}/comments?limit={1}&access_token={2}'.format(
            post_id, self.retrieve_limit, self.access_token)
        data = requests.get(url).json()
        return data['data'] #取得GraphAPI中'data'資料

    def draw(self, comments, num_of_prize):
        fans = list() #取得GraphAPI中fans資料，資料型態改為list()
        user_ids = set() #取得GraphAPI中user_ids資料，資料型態改為set()
        for comment in comments: #執行for迴圈，取得GraphAPI當中comments資料
            if comment['id'] in user_ids:
                continue
            else:
                user_ids.add(comment['id'])
                fans.append({
                    'message': comment['message'],
                    'created_time': comment['created_time'],
                    'id': comment['id']
                })
        random.shuffle(fans) #引用random模組當中之shuffle函式
        draws = fans[:num_of_prize]
        with open('draw.csv', 'w', encoding='utf-8') as csvfile:
            fieldnames = ['id', 'created_time', 'message']
            writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
            writer.writeheader()
            for draw in draws:
                print(draw)
                writer.writerow(draw)


if __name__ == '__main__':
    #----------------------------下方開始為主要抽獎程式碼--------------------------------------------
    ACCESS_TOKEN = 'EAACljVOZBauUBAMvlD9jgZBkmmo0zKEjr9pBL5CdYeXlDR5vwfXvBHUqgT4JDdVrK9eIKEO2Kctetk3KVJkQNpnlVkJfV1krL41GdyQu5CY2Wvd3Go1f3UvOEQGnR2LeFxf7U7aDd7qQ4Gqrqyei4dWCJjcL4YUJG3OQujPyzMAZBqdsCxamlX6u6BXOydZCv0EoCkDhZCwZDZD'
    #東吳大學校友總會Facebook GraphAPI權杖
    fl = FbLottery('https://www.facebook.com/scuaa.tw', ACCESS_TOKEN) #存取校友總會粉絲專頁主要連結與權杖
    lottery_post = fl.get_lottery_post('校友藝起來・兩廳院看明華園') #尋找有關鍵字之目標文章(導向get_lottery_post定義函式)
    lottery_post_comments = fl.get_post_comments(lottery_post['id']) #尋找目標文章之流言(導向get_post_comments定義函式)
    fl.draw(lottery_post_comments, 10) #利用draw函數抽出10名得獎者 (導向draw定義函式)

{'message': '黃文琦，戴口罩，梅花座，進戲院，挺校友', 'created_time': '2021-09-24T10:45:39+0000', 'id': '1505909493094645_1506797766339151'}
{'message': '凡乃乃戴口罩，梅花座，進戲院，挺校友', 'created_time': '2021-09-23T22:25:05+0000', 'id': '1505909493094645_1506454273040167'}
{'message': 'Anita Yang,戴口罩， 梅花座，進戲院，挺校友。', 'created_time': '2021-09-24T00:30:36+0000', 'id': '1505909493094645_1506504179701843'}
{'message': 'Emerald Yeh  戴口罩，梅花座，進戲院，挺校友', 'created_time': '2021-09-23T15:19:51+0000', 'id': '1505909493094645_1506232743062320'}
{'message': 'Celia Hsieh 戴口罩，梅花座，進戲院，挺校友', 'created_time': '2021-09-23T14:18:08+0000', 'id': '1505909493094645_1506198413065753'}
{'message': 'Katy Yu，戴口罩，梅花座，進戲院，挺校友', 'created_time': '2021-09-23T14:19:44+0000', 'id': '1505909493094645_1506199339732327'}
{'message': '朱珮瑜 戴口罩，梅花座，進戲院，挺校友', 'created_time': '2021-09-23T14:16:30+0000', 'id': '1505909493094645_1506197633065831'}
{'message': 'Yichen Huang 戴口罩，梅花座，進戲院，挺校友~帶媽媽一起去看戲❤️', 'created_time': '2021-09-23T14:42:56+0000', 'id': '15059094930